In [1]:
import os 
import numpy as np 
import cv2 
import torch 
import random
from tqdm import tqdm

/home/asilla/anaconda3/envs/duongnh/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Update Augementation offline
path_dataset = "/home/asilla/duongnh/project/Analys_COCO/tmp_folder/CrossPseudo_UpdateBranch/Dataset/TrainDataset"
path_des_dataset = "/home/asilla/duongnh/project/Analys_COCO/tmp_folder/CrossPseudo_UpdateBranch/Dataset/Augmentation/Mosaic_augmentation"
if not os.path.exists(path_des_dataset + "/mask"):
    os.makedirs(path_des_dataset + "/mask")

if not os.path.exists(path_des_dataset + "/image"):
    os.makedirs(path_des_dataset + "/image")

In [8]:
path_image = path_dataset + "/image"
path_mask = path_dataset + "/mask"
# l_dataset_nimage = os.listdir(path_image)

file_labeled = open("dataloader/log.txt", "r")
l_dataset_nimage = []
for line_nimage in file_labeled:
    line_nimage = line_nimage[:-1]
    l_dataset_nimage.append(line_nimage)
print(len(l_dataset_nimage))



290


In [9]:
def process_image(path_image):
    image = cv2.imread(path_image)
    image = cv2.resize(image, (352, 352))
    return image

In [10]:
def merge_image(l_image):
    l_used_nimage = random.choices(l_image, k=4)
    blank_image = np.zeros((352, 352, 3), np.uint8)
    blank_mask = np.zeros((352, 352,3), np.uint8)
    l_image_used = [process_image(path_image + "/" + nimage) for nimage in l_used_nimage]
    l_mask_used = [process_image(path_mask + "/" + nmask) for nmask in l_used_nimage]
    l_resize_image = [cv2.resize(image, (176, 176)) for image in l_image_used]
    l_resize_mask = [cv2.resize(mask, (176, 176)) for mask in l_mask_used]
    blank_image[:176, :176, :] = l_resize_image[0]
    blank_image[:176, 176:, :] = l_resize_image[1]
    blank_image[176:, :176, :] = l_resize_image[2]
    blank_image[176:, 176:, :] = l_resize_image[3]
    
    blank_mask[:176, :176, :] = l_resize_mask[0]
    blank_mask[:176, 176:, :] = l_resize_mask[1]
    blank_mask[176:, :176, :] = l_resize_mask[2]
    blank_mask[176:, 176:, :] = l_resize_mask[3]
    
    return blank_image, blank_mask

In [11]:
def mosaic_augmentation(l_nimage):
    l_used_nimage = random.choices(l_nimage, k=4)
    blank_image = np.zeros((352, 352, 3), np.uint8)
    blank_mask = np.zeros((352, 352,3), np.uint8)
    l_image_used = [process_image(path_image + "/" + nimage) for nimage in l_used_nimage]
    l_mask_used = [process_image(path_mask + "/" + nmask) for nmask in l_used_nimage]
    blank_image[:176, :176, :] = l_image_used[0][:176, :176, :]
    blank_image[:176, 176:, :] = l_image_used[1][:176, 176:, :]
    blank_image[176:, :176, :] = l_image_used[2][176:, :176, :]
    blank_image[176:, 176:, :] = l_image_used[3][176:, 176:, :]
    
    blank_mask[:176, :176, :] = l_mask_used[0][:176, :176, :]
    blank_mask[:176, 176:, :] = l_mask_used[1][:176, 176:, :]
    blank_mask[176:, :176, :] = l_mask_used[2][176:, :176, :]
    blank_mask[176:, 176:, :] = l_mask_used[3][176:, 176:, :] 
    
    return blank_image, blank_mask

In [12]:
#augmentation 
NUMBER_AUGMENTATION = 290

for i in tqdm(range(NUMBER_AUGMENTATION)):
    if random.random() <= 0.5:
        blank_image, blank_mask = merge_image(l_dataset_nimage)
        name_image = "augmentaton_merge_" + str(i) + ".png"
    else:
        blank_image, blank_mask = mosaic_augmentation(l_dataset_nimage)
        name_image = "augmentaton_mosaic_" + str(i) + ".png"
    cv2.imwrite(path_des_dataset + "/image/" + name_image, blank_image)
    cv2.imwrite(path_des_dataset + "/mask/" + name_image, blank_mask)

100%|██████████| 290/290 [00:09<00:00, 29.62it/s]
